In [72]:
import pandas as pd
import numpy as np

In [82]:
rf_train = pd.read_csv('./Train submission/rf_train.csv')
gb_train = pd.read_csv('./Train submission/gb_train.csv')
dt_train = pd.read_csv('./Train submission/dt_final_train.csv')
ridge_train = pd.read_csv('./Train submission/bagging_ridge_train.csv')
svr_train = pd.read_csv('./Train submission/bagging_svr_train.csv')
xgb_train = pd.read_csv('./Train submission/xgb_train.csv')
adb_train = pd.read_csv('./Train submission/adb_train.csv')
ma_train = pd.read_csv('./Train submission/ma_train.csv')

In [83]:
print(rf_train.shape)
print(gb_train.shape)
print(dt_train.shape)
print(ridge_train.shape)
print(svr_train.shape)
print(xgb_train.shape)
print(adb_train.shape)
print(ma_train.shape)

(1494, 2)
(1494, 2)
(1494, 2)
(1494, 2)
(1494, 2)
(1494, 2)
(1494, 2)
(1494, 2)


In [131]:
# merge_train = pd.merge(ma_train, rf_train, on='ID', how='left', suffixes=('_ma', '_rf'))
merge_train = pd.merge(ma_train, gb_train, on='ID', how='left', suffixes=('_ma', '_gb'))
# merge_train = pd.merge(merge_train, dt_train, on='ID', how='left', suffixes=('_gb', '_dt'))
# merge_train = pd.merge(merge_train, ridge_train, on='ID', how='left', suffixes=('_dt', '_ridge'))
# merge_train = pd.merge(merge_train, svr_train, on='ID', how='left', suffixes=('_ridge', '_svr'))
merge_train = pd.merge(merge_train, xgb_train, on='ID', how='left', suffixes=('_gb', '_xgb'))
merge_train = pd.merge(merge_train, adb_train, on='ID', how='left', suffixes=('_xgb', '_adb'))
X = merge_train

In [132]:
y = pd.read_csv('../challenge_data/Y_train.csv')

In [133]:
ma_test = pd.read_csv('./Submission/MA_final.csv')
rf_test = pd.read_csv('./Submission/rf_test.csv')
gb_test = pd.read_csv('./Submission/gb_final.csv')
dt_test = pd.read_csv('./Submission/dt_final.csv')
ridge_test = pd.read_csv('./Submission/bagging_linear_test.csv')
svr_test = pd.read_csv('./Submission/bagging_svr_test.csv')
xgb_test = pd.read_csv('./Submission/xgb_final.csv')
adb_test = pd.read_csv('./Submission/adab_final.csv')

In [134]:
# merge_test = pd.merge(ma_test, rf_test, on='ID', how='left', suffixes=('_ma', '_rf'))
merge_test = pd.merge(ma_test, gb_test, on='ID', how='left', suffixes=('_ma', '_gb'))
# merge_test = pd.merge(merge_test, dt_test, on='ID', how='left', suffixes=('_gb', '_dt'))
# merge_test = pd.merge(merge_test, ridge_test, on='ID', how='left', suffixes=('_dt', '_ridge'))
# merge_test = pd.merge(merge_test, svr_test, on='ID', how='left', suffixes=('_ridge', '_svr'))
merge_test = pd.merge(merge_test, xgb_test, on='ID', how='left', suffixes=('_gb', '_xgb'))
merge_test = pd.merge(merge_test, adb_test, on='ID', how='left', suffixes=('_xgb', '_adb'))
X_test = merge_test

In [135]:
X = X.set_index('ID').sort_index().reset_index(drop=True)
y = y.set_index('ID').sort_index().reset_index(drop=True)
X_test = X_test.set_index('ID').sort_index().reset_index(drop=True)

In [136]:
print(X.shape)
print(X_test.shape)

(1494, 4)
(654, 4)


In [144]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC
from scipy.stats import spearmanr

In [152]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.3)
model = LinearRegression()
model.fit(x_train, y_train['TARGET'])
y_pred = model.predict(x_test)
spearman_corr_lr = spearmanr(y_pred, y_test).correlation
print(f"Spearman Correlation for Linear Regression: {spearman_corr_lr:.1%}")

Spearman Correlation for Linear Regression: 75.4%


In [154]:
# 获取模型的系数（权重）
coefficients = model.coef_

# 获取特征的名称
feature_names = X.columns

# 将特征名称和系数一一对应并打印
feature_weights = list(zip(feature_names, coefficients))
for feature, weight in feature_weights:
    print(f"{feature}: {weight:.4f}")

TARGET_ma: -0.0111
TARGET_gb: 0.5134
TARGET_xgb: 0.5134
TARGET_adb: 0.0728


In [146]:
from sklearn.svm import SVR

svm_model = SVR()
svm_model.fit(x_train, y_train['TARGET'])
y_pred_svm = svm_model.predict(x_test)
spearman_corr_svm = spearmanr(y_pred_svm, y_test).correlation
print(f"Spearman Correlation for SVM: {spearman_corr_svm:.1%}")

Spearman Correlation for SVM: 77.5%


In [155]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor()
rf_model.fit(x_train, y_train['TARGET'])
y_pred_rf = rf_model.predict(x_test)
spearman_corr_rf = spearmanr(y_pred_rf, y_test).correlation
print(f"Spearman Correlation for Random Forest: {spearman_corr_rf:.1%}")

Spearman Correlation for Random Forest: 72.2%


In [156]:
# Get feature importances
feature_importances = rf_model.feature_importances_

# Get feature names
feature_names = x_train.columns

# Combine feature names and feature importances and print
feature_importance_list = list(zip(feature_names, feature_importances))
feature_importance_list.sort(key=lambda x: x[1], reverse=True)  # Sort by importance (descending)

# Print feature importance
for feature, importance in feature_importance_list:
    print(f"{feature}: {importance:.4f}")

TARGET_xgb: 0.3995
TARGET_gb: 0.3838
TARGET_adb: 0.1516
TARGET_ma: 0.0651


In [148]:
from sklearn.ensemble import GradientBoostingRegressor

gb_model = GradientBoostingRegressor()
gb_model.fit(x_train, y_train['TARGET'])
y_pred_gb = gb_model.predict(x_test)
spearman_corr_gb = spearmanr(y_pred_gb, y_test).correlation
print(f"Spearman Correlation for Gradient Boosting: {spearman_corr_gb:.1%}")


Spearman Correlation for Gradient Boosting: 73.5%


In [149]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor()
xgb_model.fit(x_train, y_train['TARGET'])
y_pred_xgb = xgb_model.predict(x_test)
spearman_corr_xgb = spearmanr(y_pred_xgb, y_test).correlation
print(f"Spearman Correlation for XGBoost: {spearman_corr_xgb:.1%}")

Spearman Correlation for XGBoost: 65.5%


In [150]:
pred = svm_model.predict(X_test)
df_test = pd.read_csv('../challenge_data/X_test.csv')
test_pred = pd.DataFrame()
test_pred['ID'] = df_test['ID']
test_pred['TARGET'] = pred
test_pred.to_csv('./Submission/' + 'stacking_final' + '.csv', index=False)